# Accessing NeuroHub using the CBRAIN API

## 0. Set up boilerplate

You **must** make sure the module `httpproxy/1.0` is loaded in your environment before starting this notebook on **[Béluga JupyterHub](https://jupyterhub.beluga.computecanada.ca/)**. **Do not** load the module in your `.bashrc`, it interferes with initial login, this is needed once you have a JupyterLab instance up and running.

If you are running this notebook **anywhere else** you **must** first install the `requests` package.

In [ ]:
!pip install requests

Next, let's import libraries that are required to send HTTP requests and manipulate JSON data.


In [ ]:
import requests
import json
import getpass

We define the API endpoint. See the [CBRAIN API](https://portal.cbrain.mcgill.ca/swagger#/) for more information about how to use the API.

In [ ]:
base_url = 'https://portal.cbrain.mcgill.ca'

## 1. Connect to NeuroHub

Connect to NeuroHub with a *username/password* or a generated API *token*.  See the [NeuroHub Portal](https://neurohub.ca/portal.html) for information about how to register for an account.

### 1.b Connect with username/password

We can use username/password authentication where we `POST` our `credentials` to `/session` and get a token that will need to be sent as a parameter within each subsequent request.

In [ ]:
credentials = {
    'login': input('username'),
    'password': getpass.getpass('password')
}

session_response = requests.post(
    url = '/'.join([base_url, 'session']),
    data = credentials,
    headers = {'Accept': 'application/json'}
)

if session_response.status_code != requests.codes.ok:
    print('Login failed.')
    print(session_response)
else:
    print('Login successful.')
    session_info = session_response.json()
    cbrain_api_token = session_info['cbrain_api_token']
    user_id = str(session_info['user_id'])
    print("User ID:", user_id)

### 1.b Connect with an API token

*Alternatively*, we can generate a [NeuroHub API Token](https://portal.neurohub.ca/myaccount) via the NeuroHub Portal itself.  Copy the generated new API token and paste it below then we'll get your user ID.

In [ ]:
cbrain_api_token = "PASTE_NEUROHUB_API_TOKEN_HERE"

session_token_response = requests.post(
    url = '/'.join([base_url, 'session']),
    data = {'cbrain_api_token': cbrain_api_token},
    headers = {'Accept': 'application/json'}
)

if session_token_response.status_code != requests.codes.ok:
    print('Session not valid.')
    print(session_token_response)
else:
    print('Session valid.')
    session_info = session_token_response.json()
    user_id = str(session_info['user_id'])
    print("User ID:", user_id)

## 2. User files

### 2.a Get all user-accessible files 

***WARNING:*** This step may take many minutes. You may want to skip this and the following subsection.

We `GET` our `request` from `/userfiles` for a list of all the files we have access to.  Notice that we page through the results, requesting a subsequent page until done.

In [ ]:
try:
    user_id = str(session_info['user_id'])
    cbrain_api_token = session_info['cbrain_api_token']
except NameError:
    raise UserWarning('Login first.')
 
files = []
files_request = {
    'cbrain_api_token': cbrain_api_token,
    'page': 1,
    'per_page': 1000
}

while True:

    files_response = requests.get(
        url = '/'.join([base_url, 'userfiles']),
        data = files_request,
        headers = {'Accept': 'application/json'}
    )

    if files_response.status_code != requests.codes.ok:
        print('User files request failed.')
        print(files_response)
        break

    # Collect the responses on this page then increment
    files += files_response.json()
    files_request['page'] += 1
    
    # Stop requesting responses when we're at the last page
    if len(files_response.json()) < files_request['per_page']:
        break 
    
print("{} files found\n".format(str(len(files))))

### 2.b Filter files
The list of files above can be filtered on any of their properties.  Here we filter for a name that contains the `.nii.gz` extension.

In [ ]:
filter_results = list(filter(lambda f: '.nii.gz' in f['name'], files))
print("{} filtered results found".format(str(len(filter_results))))

Let's arbitrarily get the first of these files.

In [ ]:
file_info = filter_results[0]
file_id = str(file_info['id'])
print(json.dumps(file_info, indent=4))

### 2.c Get a single file

*Alternatively*, we can get the file directly if we know it's file ID. You can get the file ID by navigating the the file info page on the NeuroHub portal and note the digits in the URL: https://portal.neurohub.ca/userfiles/FILE_ID

In [ ]:
file_id = "PASTE_FILE_ID_HERE"

file_response = requests.get(
    url = '/'.join([base_url, 'userfiles', file_id]),
    data = {'cbrain_api_token': cbrain_api_token},
    headers = {'Accept': 'application/json'}
)

file_info = file_response.json()
print(json.dumps(file_info, indent=4))

### 2.d Get file contents

We `GET` the user file content from `/userfiles/{id}/content` for the first filtered result.

In [ ]:
file_content_response = requests.get(
    url = '/'.join([base_url, 'userfiles', file_id, 'content']),
    data = {'cbrain_api_token': cbrain_api_token},
    headers = {'Accept': 'application/json'}
)

# What did I forget here?

print(file_content_response.headers)
file_content = file_content_response.content

### 2.e Process data

You can now do any interactive processing with `response.content`.

Let's compare the computed size to the stated size.

In [ ]:
print(len(file_content))
print(file_info['size'])

## 3. Logout

Destroy the current session ans API Token by sending a `DELETE` to `/session`.  You'll have to go back to step 1 to login with a username/password or generate a new API Token.

In [ ]:
session_destroy_response = requests.delete(
    url= '/'.join([base_url, 'session']),
    data = {'cbrain_api_token': cbrain_api_token},
    headers = {'Accept': 'application/json'}
)

if session_destroy_response.status_code == requests.codes.ok:
    print("Session ended.")
else:
    print("Session failure.")